## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers, Input

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime',
             'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
             'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Split the data into training and testing sets
# from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_cat_cols = X_train.select_dtypes(include='object').columns.tolist()
X_encoder = OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown='ignore')
X_encoder.fit(X_train[X_cat_cols])

X_train_encoded = pd.DataFrame(X_encoder.transform(X_train[X_cat_cols]), index=X_train.index)
X_test_encoded = pd.DataFrame(X_encoder.transform(X_test[X_cat_cols]), index=X_test.index)

X_train = X_train.drop(columns=X_cat_cols)
X_test = X_test.drop(columns=X_cat_cols)
X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test = pd.concat([X_test, X_test_encoded], axis=1)

# Check the count of each value in the 'OverTime' column
attrition_df["OverTime"].value_counts()


OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [7]:
# Ensure all column names are strings
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the testing data
X_test_scaled = scaler.transform(X_test)

In [8]:
print("Final shape of X_train_scaled:", X_train_scaled.shape)


Final shape of X_train_scaled: (1102, 10)


In [9]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
dept_encoder.fit(y_train[["Department"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept = dept_encoder.transform(y_train[["Department"]])
y_test_dept = dept_encoder.transform(y_test[["Department"]])

# Print encoded department values to visualize the result
print(y_train_dept)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [10]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
attr_encoder.fit(y_train[["Attrition"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr = attr_encoder.transform(y_train[["Attrition"]])
y_test_attr = attr_encoder.transform(y_test[["Attrition"]])

# Print encoded attrition values to visualize the result
print(y_train_attr)


[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


## Part 2: Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data.
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = Input(shape=(input_dim,), name='input')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)


In [12]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_branch = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer
dept_output = layers.Dense(y_train_dept.shape[1], activation='softmax', name='department_output')(dept_branch)

In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_branch = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attr_output = layers.Dense(y_train_attr.shape[1], activation='softmax', name='attrition_output')(attr_branch)

In [14]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attr_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output': 'categorical_crossentropy',
        'attrition_output': 'categorical_crossentropy'
    },
    metrics={
        'department_output': 'accuracy',
        'attrition_output': 'accuracy'
    }
)

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 10)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │        704 │ input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Train the model
# Train the model using the preprocessed data
history = model.fit(
    X_train_scaled,
    {'department_output': y_train_dept, 'attrition_output': y_train_attr},
    epochs=100,
    batch_size=32,
    validation_split=0.2,  # Optional, gives a validation curve
    verbose=1
)


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - attrition_output_accuracy: 0.8469 - attrition_output_loss: 0.5054 - department_output_accuracy: 0.6958 - department_output_loss: 0.8611 - loss: 1.3667 - val_attrition_output_accuracy: 0.8281 - val_attrition_output_loss: 0.4668 - val_department_output_accuracy: 0.6335 - val_department_output_loss: 0.8391 - val_loss: 1.3067
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8318 - attrition_output_loss: 0.4120 - department_output_accuracy: 0.6613 - department_output_loss: 0.7708 - loss: 1.1828 - val_attrition_output_accuracy: 0.8281 - val_attrition_output_loss: 0.4818 - val_department_output_accuracy: 0.6335 - val_department_output_loss: 0.8572 - val_loss: 1.3394
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8404 - attrition_output_loss: 0.3784 - department_output_accuracy: 0.6809 - department_output_loss: 0.7248 - loss: 1.1034 - val_attrition_output_accuracy: 0.8281 -

In [16]:
# Evaluate the model with the testing data
results = model.evaluate(
    X_test_scaled,
    {'department_output': y_test_dept, 'attrition_output': y_test_attr},
    verbose=0
)
print("Results:", results)
print("Metrics:", model.metrics_names)


Results: [5.173780918121338, 3.042943239212036, 2.0333712100982666, 0.79347825050354, 0.5461956262588501]
Metrics: ['loss', 'compile_metrics', 'department_output_loss', 'attrition_output_loss']


In [18]:
# Extract accuracy values from the results list
dept_acc = results[3]  # Department accuracy
attr_acc = results[4]  # Attrition accuracy

# Print the accuracy for both department and attrition
print("Attrition predictions accuracy:", round(attr_acc, 4))
print("Department predictions accuracy:", round(dept_acc, 4))

Attrition predictions accuracy: 0.5462
Department predictions accuracy: 0.7935


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is helpful, but if the classes are imbalanced (like very few people leaving), it can be misleading. Other metrics like precision or recall might give a clearer picture.
2. I used softmax for the output layers because both targets are categories, and softmax gives us probabilities for each class.
3. The model could be improved by adding dropout, trying different architectures, or tuning things like the number of layers and batch size.